# Generische Programmierung mit Generics

#### Marcel Lüthi <br/> Departement Mathematik und Informatik

# Motivation

* Bisherige Lösung um Klassen für mehrere Typen verwendbar zu machen:
    * Nutzung von Supertyp Objekt

In [1]:
class Stack { 
    
    private Object[] data = new Object[1000]; // Vereinfachung: Feste grösse
    private int nElements = 0; // Anzahl Elemente im Stack
 
    public void push(Object element) {
        data[nElements] = element;     
        nElements += 1;
    }
    
    public Object pop() { 
        if (nElements <= 0) {
            return null;
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}

# Probleme

* Wir brauchen explizite downcasts
* Intent nicht klar
    * Speichern wir Objekte oder Listen oder strings?
* Nicht typesafe
    * Fehler zur Laufzeit - Keine Hilfe von Compiler
* Klassen müssen in Hierarchie gebracht werden
    * Nicht möglich ohne Source Code

In [29]:
Stack stack = new Stack();
stack.push("abc");
stack.push(5);

String s = (String) stack.pop();

EvalException: java.base/java.lang.Integer cannot be cast to java.base/java.lang.String

# Lösung: Java Generics

In [30]:
class Stack<E> { 
    
    private E[] data = (E[])new Object[1000];
    private int nElements = 0; // Anzahl Elemente im Stack
 
 
    public void push(E element) {
        data[nElements] = element;     
        nElements += 1;
    }
    
    public E pop() { 
        if (nElements <= 0) {
            return null;
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}

# Nutzung des Generischen Stacks

In [22]:
Stack<Integer> intStack = new Stack<Integer>();
intStack.push(5);
Integer a = intStack.pop();


### Alternative Syntax

In [ ]:
Stack<Integer> intStack = new Stack<>();

#### Vorteile 

* Homogene Datenstruktur mit Compilezeit Typprüfung
* Typsicherheit (keine Casts)

# Terminologie

<div style="width:60%">
<img src="images/generics-terminology.png"/>
</div>

# Mehrere Platzhalter

In [21]:
class Map<K, V> {

    K[] keys = (K[]) new Object[1000];
    V[] values = (V[]) new Object[1000];
    int nElements = 0;

    void put(K key, V value) {
        keys[nElements] = key;
        values[nElements] = value;
        nElements += 1;
    }

    V get(K key) {
        for (int i = 0; i < nElements; ++i) { 
            if (keys[i].equals(key)) { 
                return values[i];
            }
        }
        return null;
    }

}

# Anwendung von Map: Geburtstage speichern

In [27]:
Map<String, Date> map = new Map<>();
map.put("sonja", new Date(10, 02, 2000));
map.put("tom", new Date(10, 02, 1997));
map.put("leslie", new Date(7, 05, 1999));

Date tomsBirthday = map.get("tom");

# Type erasure

* Generische Typen sind nur zur Kompilationszeit bekannt

![erasure](images./generics-erasure.png)

# Erzeugen von Objekten von Generischem Typ

> Nicht möglich!

* Keine Typinformation zur Laufzeit verfügbar.

In [99]:
class Foo<T> {
    T t = new T();
}

CompilationException: 

* Hack für Arrays 

```E[] data = (E[])new Object[1000];```

* Saubere Alternative: Factory


# Factories

* Factory sind einfache Klassen um Objekte zu erzeugen.
* Saubere Alternative um Objekte von generischem Typ zu erzeugen
* Verbreitetes Muster in der Softwareentwicklung 

In [66]:
interface Factory<T> { 
    public T createInstance();
}


# Factory Implementation

In [67]:
class IntegerFactory implements Factory<Integer> { 
    int value = 0; 
    public IntegerFactory(int i)  {
        this.value = value;
    }
    public Integer createInstance() { 
        return new Integer(value);
    }
}

* Entsprechend auch für Strings, Dates, ....

# Nutzung der Factory

In [57]:
class Foo<T> {

    T t = null;
    
    public Foo(Factory<T> factory) { 
        t = factory.createInstance();
    }
}

In [63]:
Foo<Integer> f = new Foo<>(new IntegerFactory(0));

# Typeinschränkungen 

# Typeinschränkungen (Bounded type parameters)

* Wir können Typeinschränkung auf generischen Parameter definieren
* Beispiel: Typ sollte vergleichbar sein: ```class A<E extends Comparable>```
    * Nur Subtypen von Comparable können als generische Typen benutzt werden.     
* Mehrere Einschränkungen sind möglich
* Beispiel: Typ sollte vergleichbar und serialisierbar sein: ```class A<E extends Comparable & Serializable>```

# Beispiel

In [88]:
class SortedList <E extends Comparable> {
      E[] data = (E[]) new Comparable[1000];
      int nElements = 0;

      void add(E elem) {
          int i = nElements - 1;
          while (i >= 0 && elem.compareTo(data[i]) > 0) {  
              data[i+1] = data[i];
              i -= 1;  
          }
          data [i+1] = elem; 
          nElements++;
     }
}

# Instanzierung 

Folgendes funktioniert:

In [96]:
new SortedList<Integer>();
new SortedList<String>();
new SortedList<Comparable>();

REPL.$JShell$96D$SortedList@5fde2c20

Folgendes funktioniert nicht:

In [98]:
new SortedList<Stack> ();
new SortedList<Object>();

CompilationException: 

# Vererbung